In [1]:
import pandas as pd
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
import numpy as np

In [2]:
data = StringIO('''Мотоцикл;Цена, руб;Год выпуска;Пробег, км;Объем двигателя, см3;Мощность двигателя, л.с.
BMW R Nine T; 1099000.00; 2017; 1070; 1200; 125
Harley-Davidson V-Rod; 5300000.00; 2015; 1; 1250; 209
Ducati 1299 Panigale; 1399000.00; 2016; 1600; 1299; 205
BMW R 1200 GS; 1305000.00; 2017; 4583; 1170; 125
Triumph Tiger Explorer; 1270000.00; 2018; 2900; 1200; 141
Коэф. знач; 0.33; 0.07; 0.25; 0.13; 0.22''')
df = pd.read_csv(data, sep=";").set_index('Мотоцикл')

In [3]:
originalDataLen = len(df) - 1
df.loc['max'] = (df.head(5).max(axis=0))
df.loc['min'] = (df.head(5).min(axis=0))
df

,"Цена, руб",Год выпуска,"Пробег, км","Объем двигателя, см3","Мощность двигателя, л.с."
Мотоцикл,,,,,
BMW R Nine T,1099000.00,2017.00,1070.00,1200.00,125.00
Harley-Davidson V-Rod,5300000.00,2015.00,1.00,1250.00,209.00
Ducati 1299 Panigale,1399000.00,2016.00,1600.00,1299.00,205.00
BMW R 1200 GS,1305000.00,2017.00,4583.00,1170.00,125.00
Triumph Tiger Explorer,1270000.00,2018.00,2900.00,1200.00,141.00
Коэф. знач,0.33,0.07,0.25,0.13,0.22
max,5300000.00,2018.00,4583.00,1299.00,209.00
min,1099000.00,2015.00,1.00,1170.00,125.00


### Выбор критериев которые максимизируем и минимизируем  
  
Правило выбора таково:  
     1. если критерий имеет прямую зависимость с качеством товара <b>(критерий растет - качество растет), то МАКСИМИЗИРУЕМ</b>  
     2. если критерий имеет обратную зависимость с качеством товара <b>(критерий падает - качество растёт), то МИНИМИЗИРУЕМ</b>  

In [4]:
maximizationMinimizationParametrDict = {}
print('"+" - максимизируем; "-" - минимизируем.')
for columnName in df.columns:
    maximizationMinimizationParametrDict[columnName] = \
        input(f'Критерий: "{columnName}", введите +/-: ')

"+" - максимизируем; "-" - минимизируем.
Критерий: "Цена, руб", введите +/-: -
Критерий: "Год выпуска", введите +/-: +
Критерий: "Пробег, км", введите +/-: -
Критерий: "Объем двигателя, см3", введите +/-: +
Критерий: "Мощность двигателя, л.с.", введите +/-: +


In [5]:
normalizedType1DF = df.drop(index=df.index[df.index.size-3:df.index.size])
normalizedType2DF = df.drop(index=df.index[df.index.size-3:df.index.size])
for columnName in df.columns:
    
    denominator = df[columnName]['max'] - df[columnName]['min']
    
    if maximizationMinimizationParametrDict[columnName] == '+':
        normalizedType1DF[columnName].iloc[0:originalDataLen] = \
            df[columnName].iloc[0:originalDataLen] / df[columnName]['max']
        
        normalizedType2DF[columnName].iloc[0:originalDataLen] = \
            (df[columnName].iloc[0:originalDataLen] - df[columnName]['min']) / denominator
    else:
        normalizedType1DF[columnName].iloc[0:originalDataLen] = \
            1 - df[columnName].iloc[0:originalDataLen] / df[columnName]['max']
        
        normalizedType2DF[columnName].iloc[0:originalDataLen] = \
            (df[columnName]['max'] - df[columnName].iloc[0:originalDataLen]) / denominator


# нормализуем матрицу

In [6]:
normalizedType2DF # чем больше - тем лучше

,"Цена, руб",Год выпуска,"Пробег, км","Объем двигателя, см3","Мощность двигателя, л.с."
Мотоцикл,,,,,
BMW R Nine T,1.000000,0.666667,0.766696,0.232558,0.000000
Harley-Davidson V-Rod,0.000000,0.000000,1.000000,0.620155,1.000000
Ducati 1299 Panigale,0.928588,0.333333,0.651026,1.000000,0.952381
BMW R 1200 GS,0.950964,0.666667,0.000000,0.000000,0.000000
Triumph Tiger Explorer,0.959295,1.000000,0.367307,0.232558,0.190476


# создаем матрицы попарных сравнений

In [36]:
таблицыПараметров = []
df = normalizedType2DF.T

diffs = []
for parametrNum in df.index:
    столбцы = df.columns
    таблица = pd.DataFrame(
        data = [], 
        columns =  pd.Index(df.columns, name=parametrNum), 
        index = pd.Index(df.columns.values) )
    таблицыПараметров.append(таблица)
    
    for k in range(0, len(df.columns)):
        столбец_k = столбцы[k]
        for j in range(0, len(df.columns)):
            if not (таблица.iat[k,j] is np.nan): continue
            
            столбец_j = столбцы[j]
            
            diff = (df[столбец_k][parametrNum] - df[столбец_j][parametrNum])
            # преимущество k-того над j-тым
            if diff < 0: continue
            diffs.append(diff)
            # if `borders` is undefined, run this codeblock to fill diffs list
            # and than analyse diffs distribution and create borders list
            # continue
            for i in range(len(borders)): # [0.0, 0.05, 0.21, 0.33, 0.37, 0.63, 0.76, 0.95, 1.0]
                if diff <= borders[i]: # i =  0,   1,    2,    3,    4,    5,    6,    7,    8
                    таблица.iat[k,j] = (i+1) # чем больше преимущество, тем больше i, тем больше рейтинг
                    таблица.iat[j,k] = 1/(i+1)
                    break
                    

# анализ выборки разниц, чтобы настроить коэффициенты границ критериев для сравнения

In [37]:
diffs.sort()
diffsDF = pd.DataFrame(diffs, columns=['Разница'])
display(diffsDF.describe().T)
#print(f'35-й процентиль выборки: {np.percentile(diffs, 35)}')
#print(f'95-й процентиль выборки: {np.percentile(diffs, 95)}')

borders = []
for border in np.linspace(36,95,9):
    borders.append(np.percentile(diffs, border))
    print(f'{border}-й процентиль выборки: {borders[-1]:.2f}')

,count,mean,std,min,25%,50%,75%,max
Разница,75.0,0.322758,0.367426,0.0,0.0,0.190476,0.658846,1.0


36.0-й процентиль выборки: 0.00
43.375-й процентиль выборки: 0.05
50.75-й процентиль выборки: 0.21
58.125-й процентиль выборки: 0.33
65.5-й процентиль выборки: 0.37
72.875-й процентиль выборки: 0.63
80.25-й процентиль выборки: 0.76
87.625-й процентиль выборки: 0.95
95.0-й процентиль выборки: 1.00


# 1.2-1.3 нормализация таблиц, вычисления средних и вывод на экран

In [38]:
for таблица in таблицыПараметров:
    таблица = таблица / таблица.sum()
    таблица['Среднее'] = таблица.mean(axis=1)
    display(таблица)

"Цена, руб",BMW R Nine T,Harley-Davidson V-Rod,Ducati 1299 Panigale,BMW R 1200 GS,Triumph Tiger Explorer,Среднее
BMW R Nine T,0.439024,0.25,0.369231,0.453782,0.486486,0.399705
Harley-Davidson V-Rod,0.04878,0.027778,0.015385,0.016807,0.027027,0.027155
Ducati 1299 Panigale,0.146341,0.222222,0.123077,0.07563,0.121622,0.137778
BMW R 1200 GS,0.146341,0.25,0.246154,0.151261,0.121622,0.183075
Triumph Tiger Explorer,0.219512,0.25,0.246154,0.302521,0.243243,0.252286


Год выпуска,BMW R Nine T,Harley-Davidson V-Rod,Ducati 1299 Panigale,BMW R 1200 GS,Triumph Tiger Explorer,Среднее
BMW R Nine T,0.135266,0.25,0.246154,0.135266,0.120921,0.177521
Harley-Davidson V-Rod,0.019324,0.035714,0.015385,0.019324,0.067179,0.031385
Ducati 1299 Panigale,0.033816,0.142857,0.061538,0.033816,0.086372,0.07168
BMW R 1200 GS,0.135266,0.25,0.246154,0.135266,0.120921,0.177521
Triumph Tiger Explorer,0.676329,0.321429,0.430769,0.676329,0.604607,0.541892


"Пробег, км",BMW R Nine T,Harley-Davidson V-Rod,Ducati 1299 Panigale,BMW R 1200 GS,Triumph Tiger Explorer,Среднее
BMW R Nine T,0.177778,0.146716,0.319392,0.266667,0.32967,0.248045
Harley-Davidson V-Rod,0.711111,0.586865,0.532319,0.3,0.384615,0.502982
Ducati 1299 Panigale,0.059259,0.117373,0.106464,0.233333,0.21978,0.147242
BMW R 1200 GS,0.022222,0.065207,0.015209,0.033333,0.010989,0.029392
Triumph Tiger Explorer,0.02963,0.083838,0.026616,0.166667,0.054945,0.072339


"Объем двигателя, см3",BMW R Nine T,Harley-Davidson V-Rod,Ducati 1299 Panigale,BMW R 1200 GS,Triumph Tiger Explorer,Среднее
BMW R Nine T,0.061538,0.022222,0.081818,0.166667,0.061538,0.078757
Harley-Davidson V-Rod,0.369231,0.133333,0.109091,0.25,0.369231,0.246177
Ducati 1299 Panigale,0.492308,0.8,0.654545,0.375,0.492308,0.562832
BMW R 1200 GS,0.015385,0.022222,0.072727,0.041667,0.015385,0.033477
Triumph Tiger Explorer,0.061538,0.022222,0.081818,0.166667,0.061538,0.078757


"Мощность двигателя, л.с.",BMW R Nine T,Harley-Davidson V-Rod,Ducati 1299 Panigale,BMW R 1200 GS,Triumph Tiger Explorer,Среднее
BMW R Nine T,0.043478,0.06015,0.033019,0.043478,0.02,0.040025
Harley-Davidson V-Rod,0.391304,0.541353,0.59434,0.391304,0.48,0.47966
Ducati 1299 Panigale,0.391304,0.270677,0.29717,0.391304,0.42,0.354091
BMW R 1200 GS,0.043478,0.06015,0.033019,0.043478,0.02,0.040025
Triumph Tiger Explorer,0.130435,0.067669,0.042453,0.130435,0.06,0.086198


Наивысший средний балл по критерию «Цена» имеет модель BMW R Nine
T, по критерию «Год выпуска» Triumph Tiger, по критерию «Пробег»
Harley-Davidson V-Rod, по критерию «Объем двигателя» Ducati 1299
Panigale, по критерию «Мощность двигателя» Harley-Davidson V-
Rod